In [139]:
import requests
import json
import numpy as np
from sklearn.model_selection import train_test_split

In [140]:
WIT_API_HOST = 'https://api.wit.ai'
WIT_API_VERSION = '20200513'

In [141]:
with open('token.txt') as file:
    TOKEN = file.read()
    
HEADERS = {
    'authorization': 'Bearer ' + TOKEN,
    'accept': 'application/vnd.wit.' + WIT_API_VERSION + '+json'
}

In [142]:
def getQueries():
    rsp = requests.get(WIT_API_HOST + '/utterances', headers=HEADERS, params={'limit': 1000})
    return rsp.json()

In [143]:
def delQueries(queries):
    if not queries:
        print('empty list, aborting')
        return
    confirm = input('CONFIRM DELETION: ')
    if confirm != 'yes':
        print('aborting delete')
        return
    print('deleting the following. \n')
    print(queries)
    
    queriesDelete = [{k: v for k, v in query.items() if k.startswith('text')} for query in queries]
    rsp = requests.delete(WIT_API_HOST + '/utterances', headers=HEADERS, data=str(queriesDelete))
    
    print(rsp.json())

In [144]:
# [a,b,c]
def getExamples(filename):
    examples = np.loadtxt(filename, dtype='str', delimiter='\n')
    return examples

In [145]:
# [[a,x],[b,x],[c,x]]
def labelExamples(examples, label):
    reshaped = np.reshape(examples, (-1, 1))
    labels = np.full(examples.shape[0], label).reshape(-1, 1)
    return np.concatenate((reshaped, labels), 1)

In [146]:
# [ task1 [[a,x],[b,x],[c,x]] , task2 [[d,y],[e,y],[f,y]] ]
def getLabeledExamples(filenames, labels):
    data = []
    for i in range(0, len(filenames)):
        labeled = labelExamples(getExamples(filenames[i]), labels[i])
        data.append(labeled)
    return data

In [147]:
# [ train: task1 + task2 , test: task1 + task2 ]
def formTrainTest(data):
    trains = []
    tests = []
    for dataset in data:
        a, b = train_test_split(dataset, test_size=0.5)
        trains.append(a)
        tests.append(b)
        
    train = np.empty((0,2))
    test = np.empty((0,2))
    for i in range(len(trains)):
        train = np.concatenate((train, trains[i]))
        test = np.concatenate((test, tests[i]))
        
    return train, test

In [148]:
def examplesToJson(examples):
    json = []
    for example in examples:
        json.append({'text': example[0], 'intent': example[1], 'entities': [], 'traits': []})
    return json

In [149]:
data = getLabeledExamples(['class.txt', 'reg.txt'], ['task_class', 'task_reg'])
train, test = formTrainTest(data)
trainJson = examplesToJson(train)
testJson = examplesToJson(test)

In [151]:
str(testJson)

"[{'text': 'decide what kind of animal this is', 'intent': 'task_class', 'entities': [], 'traits': []}, {'text': 'build a model that knows whether a medical procedure will succeed', 'intent': 'task_class', 'entities': [], 'traits': []}, {'text': 'detect if a patient has bronchitis from xrays', 'intent': 'task_class', 'entities': [], 'traits': []}, {'text': 'perform facial recognition ', 'intent': 'task_class', 'entities': [], 'traits': []}, {'text': 'model which identifies contaminants', 'intent': 'task_class', 'entities': [], 'traits': []}, {'text': 'label news as real or fake', 'intent': 'task_class', 'entities': [], 'traits': []}, {'text': 'create something that predicts if a crime will occur', 'intent': 'task_class', 'entities': [], 'traits': []}, {'text': 'determines the level of security of a codebase', 'intent': 'task_class', 'entities': [], 'traits': []}, {'text': 'automated detection of safe drinking water', 'intent': 'task_class', 'entities': [], 'traits': []}, {'text': 'know

In [150]:
rsp = requests.post(WIT_API_HOST + '/utterances', headers=HEADERS, data=str(trainJson))
rsp.json()

{'sent': True, 'n': 27}

In [159]:
for testExample in testJson:
    rsp = requests.get(WIT_API_HOST + '/message', headers=HEADERS, params={'q': testExample['text']})
    rspJson = rsp.json()
    print(rspJson)
    print(rspJson['intents'][0]['name'] == testExample['intent'])
    print()

{'text': 'decide what kind of animal this is', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.9131}], 'entities': {}, 'traits': {}}
True

{'text': 'build a model that knows whether a medical procedure will succeed', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.9822}], 'entities': {}, 'traits': {}}
True

{'text': 'detect if a patient has bronchitis from xrays', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.9937}], 'entities': {}, 'traits': {}}
True

{'text': 'perform facial recognition', 'intents': [{'id': '184248982902938', 'name': 'task_reg', 'confidence': 0.6769}], 'entities': {}, 'traits': {}}
False

{'text': 'model which identifies contaminants', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.9586}], 'entities': {}, 'traits': {}}
True

{'text': 'label news as real or fake', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.6274}], 'en

In [ ]:
#getQueries()

In [ ]:
#delQueries(getQueries())